In [1]:
import sys
import os
sys.path.insert(0,os.getcwd()+"/../torch-ngp")

In [2]:
from nerf.network_tcnn import NeRFNetwork
from nerf.provider import NeRFDataset
from easydict import EasyDict
from nerf.gui import NeRFGUI
from nerf.utils import *
import torch

In [3]:
model = NeRFNetwork(
    encoding="hashgrid", encoding_dir="sphere_harmonics", 
    num_layers=2, hidden_dim=64, geo_feat_dim=15, num_layers_color=3, hidden_dim_color=64, 
    cuda_ray=True,
).eval()

In [15]:
opt = EasyDict()
opt.gui = False
opt.mode = "colmap"
opt.path = "../data/lego"
opt.W = 800
opt.H = 800
opt.scale = 0.33
opt.bound = 3

In [16]:
train_dataset = NeRFDataset(opt.path, type='val', mode=opt.mode, scale=opt.scale, preload=False)
trainer = Trainer('ngp', vars(opt), model, workspace="../workspace/lego", optimizer=None, criterion=None, ema_decay=0.95, fp16=True, lr_scheduler=None)

[INFO] Trainer: ngp | 2022-04-07_16-50-47 | cuda:0 | fp16 | ../workspace/lego

[INFO] #parameters: 12206480

[INFO] Loading latest checkpoint ...

[INFO] Latest checkpoint is ../workspace/lego/checkpoints/ngp_ep0001.pth.tar

[INFO] loaded model.

[WARN] Failed to load optimizer, use default.

[WARN] Failed to load scheduler, use default.

[INFO] loaded scaler.

In [17]:
sample = train_dataset[0]
pose = sample["pose"]
intr = sample["intrinsic"]
focal = intr[0,0].item()
H,W = sample["image"].shape[-3:-1]

In [20]:
H

800

In [21]:
res = trainer.test_gui(pose/3,intr,W,H)
print(res["depth"].min().item(),res["depth"].max().item())

0.0 0.0


In [ ]:

fig, plots = plt.subplots(1,2,figsize=(20,10))

plots[0].matshow(res["image"])
plots[1].matshow(res["depth"])
plt.show()